Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Abstractive Summarization on CNN/DM Dataset using Transformers


### Summary

This notebook demonstrates how to fine tune Transformers models like [BART](https://arxiv.org/abs/1910.13461) and [T5](https://arxiv.org/abs/1910.10683) together with HuggingFace's [transformers library](https://github.com/huggingface/transformers)for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.




### Before You Start

Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = True, the notebook takes about 5 minutes to run on a VM with 1 Tesla K80 GPUs with 12GB GPU memory. If QUICK_RUN = False, it takes around 15 minutes for data preprocessing, 15 minutes for fine-tuning and 3 hours for running evaluation on the whole CNN/DM test dataset.

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [1]:
%load_ext autoreload

%autoreload 2
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True

### Configuration


In [ ]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import time
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

# from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.dataset.swiss import SwissSummarizationDataset

from utils_nlp.models.transformers.abstractive_summarization_bartt5 import (
    AbstractiveSummarizer)

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint
start_time = time.time()


### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For abstractive summarization, the following pretrained models are supported. 

In [ ]:
pd.DataFrame({"model_name": AbstractiveSummarizer.list_supported_models()})

In [ ]:
# Transformer model being used
# MODEL_NAME = "bart-large"
MODEL_NAME = "airKlizz/t5-base-multi-de-wiki-news"
MODEL_PATH_NAME = MODEL_NAME.replace("/","_")
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = "./"+ MODEL_PATH_NAME + "_cache" #TemporaryDirectory().name
summarizer = AbstractiveSummarizer(MODEL_NAME, cache_dir=CACHE_DIR)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.   The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.


In [ ]:
validation = False
DATA_NAME ="swiss"


# the data path used to save the downloaded data file
DATA_PATH = "./"+ MODEL_PATH_NAME + "_" + DATA_NAME #TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [ ]:



if DATA_NAME is "cnndm":
    train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH)
elif DATA_NAME is "swiss":
    if validation:
        train_dataset, validation_dataset, test_dataset = SwissSummarizationDataset(top_n=TOP_N, validation=True, language='german')
    else:
        train_dataset, test_dataset = SwissSummarizationDataset(top_n=TOP_N, validation=False, language='german')
        

In [ ]:
train_dataset_concat = []
test_dataset_concat = []

for d in range(len(train_dataset)):
    train_dataset_concat.append({
        "src":(" ").join(train_dataset[d]["src_txt"]),
        "tgt":(" ").join(train_dataset[d]["tgt_txt"]),
    })


for d in range(len(test_dataset)):
    test_dataset_concat.append({
        "src":(" ").join(test_dataset[d]["src_txt"]),
        "tgt":(" ").join(test_dataset[d]["tgt_txt"]),
    })

Preprocess the data.

In [ ]:
from multiprocessing import Pool
def preprocess(summarizer, input_data_list, num_workers=50, chunk_size=100, internal_batch_size=5e3):
        """ preprocess the data for abstractive summarization.

        Args:
            input_data_list (list of dictionary): input list where each item is
                an dictionary with fields "src" and "tgt" and both fields are string.
            num_workers (int, optional): The number of workers in the pool.
                Defautls to 50.
            chunk_size (int, optional): The size that a worker processes.
                Defaults to 100.
            internal_batch_size (int, optional): The size that one pool processes.
                Defaults to 5000. Reduce this number if you see segment fault.

        Returns:
            list of dictionary with addtional fields "source_ids",
                "source_mask" and "target_ids".
        """
        i = 0
        temp_dir = TemporaryDirectory().name
        os.makedirs(temp_dir, mode=0o777, exist_ok=False)
        temp_file = ".temp_preprocess"
        processed_length = 0
        result = []
        print(len(input_data_list))
        pool = Pool(num_workers, initializer=summarizer.processor.initializer)
        while processed_length < len(input_data_list):
            max_length = int(min(processed_length+internal_batch_size, len(input_data_list)))
            temp = []
            for j in range(processed_length, max_length):
                temp.append(input_data_list[j])
            result_generator = pool.imap(summarizer.processor.encode_example, temp, chunk_size)
            torch.save(list(result_generator), os.path.join(temp_dir, temp_file+str(i)))
            i += 1
            processed_length = max_length
            #print(processed_length)

        pool.close()
        pool.join()
        result = []
        total_batch_number = i
        for i in range(total_batch_number):
            result.extend(torch.load(os.path.join(temp_dir, temp_file+str(i))))
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir, ignore_errors=True)
        return result



In [ ]:
%%time
# abs_sum_train = summarizer.processor.preprocess(train_dataset)
abs_sum_train = preprocess(summarizer, train_dataset_concat)
# torch.save(abs_sum_train,  os.path.join(DATA_PATH, "train_{0}_full.pt".format(MODEL_NAME)))
# abs_sum_train = torch.load(os.path.join(DATA_PATH, "train_{0}_full.pt".format(MODEL_NAME)))

In [ ]:
%%time
# abs_sum_test= summarizer.processor.preprocess(test_dataset)
abs_sum_test= preprocess(summarizer, test_dataset_concat)

In [ ]:
print(len(abs_sum_train))
print(len(abs_sum_test))

#### Inspect Data

In [ ]:
abs_sum_train[0].keys()

In [ ]:
abs_sum_train[5]

## Fine tune model
To start model fine-tuning, we need to specify the paramters as follows.

In [ ]:
BATCH_SIZE_PER_GPU = 3
GRADIENT_ACCUMULATION_STEPS = 1
MAX_POS_LENGTH = 512

# GPU used for training
NUM_GPUS = torch.cuda.device_count()


# Learning rate
LEARNING_RATE=3e-5
MAX_GRAD_NORM=0.1

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100
SAVE_EVERY=1000

# total number of steps for training
MAX_STEPS=100
# number of steps for warm up
WARMUP_STEPS=5e1
    
if not QUICK_RUN:
    MAX_STEPS=1000
    WARMUP_STEPS=5e2
    
# inference parameters
TEST_PER_GPU_BATCH_SIZE = 96
BEAM_SIZE = 3
 

In [ ]:

summarizer.fit(
            abs_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE_PER_GPU*NUM_GPUS,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            max_grad_norm=MAX_GRAD_NORM,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
        )


In [ ]:
# save a finetuned model and load a previous saved model
"""
import torch
model_path = os.path.join(
        CACHE_DIR,
        "abssum_modelname_{0}_steps_{1}.pt".format(
            MODEL_NAME, MAX_STEPS
        ))
summarizer.save_model(global_step=MAX_STEPS, full_name=model_path)

summarizer = AbstractiveSummarizer(MODEL_NAME, cache_dir=CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu")['model'])
"""

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.             
For the settings in this notebook with QUICK_RUN=False, you should get ROUGE scores close to the following numbers:

``
{'rouge-1': {'f': 0.3532833731474843,
             'p': 0.5062112092750258,
             'r': 0.2854026986121758},
 'rouge-2': {'f': 0.1627400891022247,
             'p': 0.23802173638805246,
             'r': 0.13034686738843493},
 'rouge-l': {'f': 0.2587374492685969,
             'p': 0.3710902340617733,
             'r': 0.20909466938819835}}
 ``           

In [ ]:
source = []
target = []

    
for i in test_dataset_concat:
    source.append(i["src"]) 
    target.append(i['tgt'].replace("<t>","").replace("</t>", "").replace("\n", "")) 

In [ ]:
%%time
prediction = summarizer.predict(abs_sum_test, 
                                num_gpus=NUM_GPUS, 
                                batch_size=TEST_PER_GPU_BATCH_SIZE*NUM_GPUS,
                                min_length=24, 
                                max_length=48,
                                num_beams=BEAM_SIZE,
                                checkpoint= "./airKlizz_t5-base-multi-de-wiki-news_cache/fine_tuned/abssum_airKlizz/t5-base-multi-de-wiki-news.pt")

In [ ]:
%%time
prediction = summarizer.predict(abs_sum_test, 
                                num_gpus=NUM_GPUS, 
                                batch_size=TEST_PER_GPU_BATCH_SIZE*NUM_GPUS,
                                min_length=24, 
                                max_length=48,
                                num_beams=BEAM_SIZE)


In [ ]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

In [ ]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

In [ ]:
prediction[3]

In [ ]:
target[3]

In [ ]:
source[0]

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [ ]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [ ]:
small_test_dataset = SummarizationDataset(
    None,
    source=[source],
    n_processes=1
)
preprocessed_dataset =  summarizer.processor.preprocess(small_test_dataset)


In [ ]:
preprocessed_dataset[0].keys()

In [ ]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, min_length=24, max_length=48,)

In [ ]:
prediction[0]

In [ ]:
print("Total notebook running time {}".format(time.time() - start_time))

## Clean up temporary folders

In [ ]:
"""
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
"""